# ORF Density Notebook

- status: finished
- implementation for evolution necessary
- could also be necessary for better positional k-mers
- no real knowledge gained!!
- discussing with manja again
- other idea:
    - Kozaki Sequence Probability (ORF Start/non-ORF Start)
    - Frameshift Probability (Start, Stop anderer Frame) 
    - All Stop/Start Positions plus Frame 
    - Prediction by prior knowledge 
        - eg. 3 proteins, 2 shifts and 1 alternative init (kozaki)
        - calculate best 3 and weight probabilitys by occurence of events
        - alternative splicing completely unimportant
        - only the protein frames with shifts and alternative inits important

![ORF](ORF.svg)

In [134]:
from Bio import SeqIO
from Bio.Seq import Seq
import regex as re
import numpy as np
import pandas as pd
import networkx as nx

In [106]:
fasta = SeqIO.parse('A.fasta','fasta')

In [107]:
entry = next(fasta)
sequence =  str(entry.seq)
header = entry.name

In [120]:
def createGraph(sequence):
    
    search = re.compile(r'ATG|TAA|TAG|TGA', flags=re.I | re.X)
    framedict = {}
    
    for match in search.finditer(sequence, overlapped=True):
        
        tripindex = match.start(0)//3
        frameindex = match.start(0)%3
        triplet = match.group(0)
        
        if tripindex not in framedict:
            framedict[tripindex] = ['N', 'N', 'N']
            framedict[tripindex][frameindex] = 'S' if triplet == 'ATG' else 'T'
        
        else:
            framedict[tripindex][frameindex] = 'S' if triplet == 'ATG' else 'T'
            
    return(framedict)

In [131]:
graph = pd.DataFrame(createGraph(sequence))

In [138]:
with open('../Desktop/test.txt', 'w') as f:
    f.write(graph.to_html())